In [17]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

proxies = {
  'http': 'http://206.189.157.23',
  'https': 'http://206.189.157.23',
}

In [18]:
olympic_json = requests.get(
             'https://en.wikipedia.org/w/api.php',
             params={
                 'action':'opensearch',
                 'search': 'summer olympic',
                 'format':'json'
             }, proxies=proxies).json()

In [19]:
olympic_json

['summer olympic',
 ['Summer Olympic Games',
  'Summer Olympic coins',
  'Summer Olympic Coins (2000–present)',
  'Summer Olympics 2020',
  'Summer Olympics 2016',
  'Summer Olympics 2012',
  'Summer Olympics 2008',
  'Summer Olympics 2004',
  'Summer Olympics medal count',
  'Summer Olympics (2000)'],
 ['', '', '', '', '', '', '', '', '', ''],
 ['https://en.wikipedia.org/wiki/Summer_Olympic_Games',
  'https://en.wikipedia.org/wiki/Summer_Olympic_coins',
  'https://en.wikipedia.org/wiki/Summer_Olympic_Coins_(2000%E2%80%93present)',
  'https://en.wikipedia.org/wiki/Summer_Olympics_2020',
  'https://en.wikipedia.org/wiki/Summer_Olympics_2016',
  'https://en.wikipedia.org/wiki/Summer_Olympics_2012',
  'https://en.wikipedia.org/wiki/Summer_Olympics_2008',
  'https://en.wikipedia.org/wiki/Summer_Olympics_2004',
  'https://en.wikipedia.org/wiki/Summer_Olympics_medal_count',
  'https://en.wikipedia.org/wiki/Summer_Olympics_(2000)']]

In [20]:
urls = olympic_json[3]

In [21]:
urls[0]

'https://en.wikipedia.org/wiki/Summer_Olympic_Games'

In [22]:
olympic_json = requests.get(
             'https://en.wikipedia.org/w/api.php',
             params={
                 'action':'opensearch',
                 'search': 'summer olympic',
                 'format':'json'
             }, proxies=proxies).json()

url = olympic_json[3][0]
page_source = requests.get(url, proxies=proxies).text
soup = BeautifulSoup(page_source, 'html.parser')

base_url = 'https://en.wikipedia.org/'
link_tags = soup.ul.find_all(name='a', href=True)
pages = soup.ul.find_all(name='a', href=True)

pages = [base_url + tag.attrs['href'] for tag in link_tags]
pages = pages[:-4]

df_info = pd.DataFrame()
df_winners_all = pd.DataFrame()
df_sports = pd.DataFrame()

for page in pages:
    yr = re.findall(r'(?:.)?(\d{4})', page)
    yr = int(yr[0])
    page_soup = BeautifulSoup(requests.get(page, proxies=proxies).text, 
                              'html.parser')
    
    #Get location, participants, events, opening/closing dates, etc.
    labels =[tag.text for tag in page_soup.find_all('th',
                                                    class_='infobox-label')]
    values = [tag.text for tag in page_soup.find_all('td',
                                                     class_='infobox-data')]
    df = pd.DataFrame()
    
    df['label'] = labels
    df['value'] = values
    df['year'] = yr
    df_info = pd.concat([df_info, df])
    
    
    # Get TOP 10 winners and rank and medals
    table = page_soup.find_all(name='table', class_=
                "wikitable sortable plainrowheaders jquery-tablesorter")
    winner = []
    rank = []
    if table:
        for tag in table[0].find_all(['a','td']):
            col = re.findall(r'rowspan="2"', str(tag))
    
            if len(col) ==0:
                winner.append(tag.text)
            
            else:
                rank.append(tag.text)
                winner.append(tag.text)
                
        winner= [i for i in winner if len(i)!=0]
        if yr in [2012, 2020]:
            winner = winner[1:]    #remove first element (error in db)
        if yr in [1904, 1936]:  
            for i in rank:

                winner.insert(int(i)*6, i)

        winners_list = []
        for i in range(10):
            winners_list.append(winner[(i*6):(1+i)*6])
            
    df_winners = pd.DataFrame(winners_list)
    df_winners.columns = ['rank', 'nation', 'gold', 'silver','bronze',
                       'total']
    df_winners['year'] = yr
    df_winners_all = pd.concat([df_winners_all, df_winners])
    
    
    #Get sports event info
    events = []
    
    if yr in [1920, 2004]:  
        dummy_list = [i.text for i in page_soup.find_all('ul' and 'li')]
        pattern = re.compile(r'\s?(\w+-?\s?\w+\s?\w+)?', flags=re.I|re.M)
        sports = []
        for i in dummy_list:
            m = pattern.search(i)
            try:
                sports.append(m.groups(0))
            except AttributeError:
                pass
            
        events = []
        if yr == 1920:
            events.append(np.unique(sports[285:310] + sports[26:59]))
        else:
            events.append(np.unique(sports[251:298]))
        events = events[0]
        
    elif yr in [1920, 1980, 1984, 1988, 1992, 1996, 2000,
                 2012, 2016, 2020]:
        event_page = page_soup.find_all('table', class_="multicol")
        event_source = event_page[0]
        event_sports= event_source.find_all('ul')

        sports = []
        for tag in event_sports:
            sports.append([i.text for i in tag.find_all('li')])

        for i in sports:
            pattern = re.compile(r'(\s?\w+-?\s?\w*)(?:\s\d)?',
                                 flags= re.I|re.M )
            for j in i:
                m = pattern.search(j)
                events.append(m.groups(0))
                
    else:
        event_source = page_soup.find(name='div', class_='div-col')
        for tag in event_source.find_all('li'):
            try:
                events.append(tag.a.text)
            except AttributeError:
                pass
            
    df_ = pd.DataFrame(events)
    df_['year'] = yr
    df_.columns = ['events', 'year']
    df_sports = pd.concat([df_sports, df_])
    

In [23]:
df_info

,label,value,year
0,Host city,"Athens, Greece",1896
1,Nations,14[1],1896
2,Athletes,241 (all men)[2],1896
3,Events,43 in 9 sports,1896
4,Opening,6 April,1896
...,...,...,...
5,Opening,23 July 2021,2020
6,Closing,8 August 2021,2020
7,Opened by,Emperor Naruhito,2020
8,Cauldron,Naomi Osaka,2020


In [24]:
df_info.tail(9)

,label,value,year
1,Motto,United by Emotion[a],2020
2,Nations,205 (+ EOR team),2020
3,Athletes,"11,090[2]",2020
4,Events,339 in 33 sports (50 disciplines),2020
5,Opening,23 July 2021,2020
6,Closing,8 August 2021,2020
7,Opened by,Emperor Naruhito,2020
8,Cauldron,Naomi Osaka,2020
9,Stadium,Olympic Stadium,2020


In [25]:
df_info.tail(30)



,label,value,year
0,Host city,"London, England, United Kingdom [1]",2012
1,Motto,Inspire a Generation,2012
2,Nations,204,2012
3,Athletes,"10,768 (5,992 men, 4,776 women)",2012
4,Events,302 in 26 sports (39 disciplines),2012
5,Opening,27 July,2012
6,Closing,12 August,2012
7,Opened by,Queen Elizabeth II[2],2012
8,Cauldron,\nCallum Airlie\nJordan Duckitt\nDesirèe Henry...,2012
9,Stadium,London Olympic Stadium,2012


In [28]:
df_info.columns

Index(['label', 'value', 'year'], dtype='object')

In [29]:
df_info

,label,value,year
0,Host city,"Athens, Greece",1896
1,Nations,14[1],1896
2,Athletes,241 (all men)[2],1896
3,Events,43 in 9 sports,1896
4,Opening,6 April,1896
...,...,...,...
5,Opening,23 July 2021,2020
6,Closing,8 August 2021,2020
7,Opened by,Emperor Naruhito,2020
8,Cauldron,Naomi Osaka,2020


In [30]:
df_winners_all



,rank,nation,gold,silver,bronze,total,year
0,1,United States,11,7,2,20,1896
1,2,Greece,10,17,19,46,1896
2,3,Germany,6,5,2,13,1896
3,4,France,5,4,2,11,1896
4,5,Great Britain,2,3,2,7,1896
...,...,...,...,...,...,...,...
5,6,Australia,17,7,22,46,2020
6,7,Netherlands,10,12,14,36,2020
7,8,France,10,12,11,33,2020
8,9,Germany,10,11,16,37,2020


In [31]:
df_sports


,events,year
0,Athletics,1896
1,Cycling,1896
2,Fencing,1896
3,Gymnastics,1896
4,Shooting,1896
...,...,...
84,Greco-Roman,2020
85,Volleyball,2020
86,Beach volleyball,2020
87,Freestyle,2020


In [32]:
olympic_date_loc = [
    'athens-1896', 'paris-1900', 'st-louis-1904', 'london-1908', 
    'stockholm-1912', 'antwerp-1920', 'paris-1924', 'amsterdam-1928',
    'los-angeles-1932', 'berlin-1936', 'london-1948', 'helsinki-1952',
    'melbourne-1956', 'rome-1960', 'tokyo-1964', 'mexico-city-1968',
    'munich-1972', 'montreal-1976', 'moscow-1980', 'los-angeles-1984',
    'seoul-1988', 'barcelona-1992', 'atlanta-1996', 'sydney-2000',
    'athens-2004', 'beijing-2008', 'london-2012', 'rio-2016'
]

In [33]:
data = []
for i in olympic_date_loc:
    olympic = requests.get('https://olympics.com/en/olympic-games/' + i,
                           proxies=proxies).text
    soup = BeautifulSoup(olympic)
    location = soup.find('h1')
    events = soup.find_all('div', 
                           class_ = 'styles__FactItems-sc-1w4me2-2 gwNkBN')
    span_text = ['Olympic Games']
    span_text_def = [location.text]
    
    for i in events:
        s_text = i.span.text
        s_text_def = i.text.replace(s_text, '')
        span_text.append(s_text)
        span_text_def.append(s_text_def)
    data.append({span_text[i]: span_text_def[i] for i in range(len(span_text))})
data

[{'Olympic Games': 'Athens1896',
  'Date': 'April 6 - April 15',
  'Country': 'Greece',
  'Athletes': '241',
  'Teams': '14',
  'Events': '43'},
 {'Olympic Games': 'Paris1900',
  'Date': 'May 14 - October 28',
  'Country': 'France',
  'Athletes': '997',
  'Teams': '24',
  'Events': '95'},
 {'Olympic Games': 'St. Louis1904',
  'Date': 'July 1 - November 24',
  'Country': 'United States',
  'Athletes': '651',
  'Teams': '12',
  'Events': '95'},
 {'Olympic Games': 'London1908',
  'Date': 'April 27 - October 31',
  'Country': 'Great Britain',
  'Athletes': '2008',
  'Teams': '22',
  'Events': '110'},
 {'Olympic Games': 'Stockholm1912',
  'Date': 'May 5 - July 27',
  'Country': 'Sweden',
  'Athletes': '2407',
  'Teams': '28',
  'Events': '102'},
 {'Olympic Games': 'Antwerp1920',
  'Date': 'April 20 - September 12',
  'Country': 'Belgium',
  'Athletes': '2622',
  'Teams': '29',
  'Events': '156'},
 {'Olympic Games': 'Paris1924',
  'Date': 'May 4 - July 27',
  'Country': 'France',
  'Athletes

In [34]:
df_olympics = pd.DataFrame(data)
df_olympics

,Olympic Games,Date,Country,Athletes,Teams,Events
0,Athens1896,April 6 - April 15,Greece,241,14,43
1,Paris1900,May 14 - October 28,France,997,24,95
2,St. Louis1904,July 1 - November 24,United States,651,12,95
3,London1908,April 27 - October 31,Great Britain,2008,22,110
4,Stockholm1912,May 5 - July 27,Sweden,2407,28,102
5,Antwerp1920,April 20 - September 12,Belgium,2622,29,156
6,Paris1924,May 4 - July 27,France,3088,44,126
7,Amsterdam1928,May 17 - August 12,Netherlands,2883,46,109
8,Los Angeles1932,July 30 - August 14,United States,1334,37,117
9,Berlin1936,August 1 - August 16,Germany,3963,49,129


In [36]:
df_olympics = pd.DataFrame(data)
a = "".join(list(df_olympics['Olympic Games'].values))
year = re.findall(r'\d+', a)
df_olympics['Year'] = year
df_olympics[['Athletes', 'Teams', 'Events', 'Year']] = (df_olympics[[
    'Athletes', 'Teams', 'Events', 'Year']].astype(int))


In [37]:
res = requests.get('https://en.wikipedia.org/'
                   'wiki/List_of_countries_by_GDP_(nominal)',
                   proxies=proxies).text
soup = BeautifulSoup(res)
pool = soup.prettify()

In [38]:
table = soup.find_all('table', class_='wikitable sortable static'
                      '-row-numbers plainrowheaders mw-datatable')

In [39]:
for country in table:
    country_list = country.find_all('a', href=True)

In [40]:
table_data = country.find_all('td')

In [41]:
lst = []
for item in country.find_all('td'):
    col = re.findall(r'colspan="2"', str(item))
    if len(col) ==0:
        lst.append(item.text)
    else:
        lst.append(None)
        lst.append(None)

In [42]:
#wrong code
table1 = []
for data in table_data:
    table1.append(data.text)


In [43]:
table2 = []
for i in range(213):
    table2.append(lst[i*8:(i+1)*8])

In [44]:
df = pd.DataFrame(table2)
df.columns=['country', 'region', 'imf_gdp','imf_year', '2020_un_gdp*',
            'un_year', 'world_bank_gdp', 'world_bank_year']
# df.drop(['imf_gdp','imf_year', 'world_bank_gdp', 'world_bank_year'])
df = df.drop(['imf_gdp','imf_year','world_bank_gdp', 'world_bank_year',
              'un_year'], axis=1)

In [45]:
df['country'] = df['country'].str.replace(" (more)", "", regex=False)

In [46]:
df

,country,region,2020_un_gdp*
0,United States,Americas,"21,433,226"
1,China,Asia,"14,342,933"
2,Japan,Asia,"5,082,465"
3,Germany,Europe,"3,861,123"
4,United Kingdom,Europe,"2,826,441"
...,...,...,...
208,Marshall Islands,Oceania,237
209,Kiribati,Oceania,194
210,Nauru,Oceania,132
211,Montserrat,Americas,67


In [47]:
url = 'https://en.wikipedia.org/wiki/2020_Summer_Olympics'

In [48]:
page_source = requests.get(url, proxies=proxies).text
soup = BeautifulSoup(page_source, 'html.parser')


In [49]:
info = soup.find_all('div', class_='div-col')
country_list = []
for tag in info:
    country_list.append(tag.text)
country= []
x = country_list[0].split('\n')

for i in x:
    i = i.replace(u'\xa0', u'')
    pattern = re.compile(r'\s?(.+)\((\d+)\)', flags=re.I)
    m = pattern.search(i)
    if m:
        country.append(m.groups(0))
        
country_dict = {}
for i,j in country:
    country_dict[i] = int(j)

In [50]:
df_part_2020 = pd.DataFrame(country_dict.items(), columns=['Country',
                                                           'Participants'])
df_part_2020['year'] =2020
df_part_2020

,Country,Participants,year
0,Afghanistan,5,2020
1,Albania,9,2020
2,Algeria,41,2020
3,American Samoa,6,2020
4,Andorra,2,2020
...,...,...,...
201,Vietnam,18,2020
202,Virgin Islands,4,2020
203,Yemen,5,2020
204,Zambia,26,2020


In [51]:
df_part_2020.sort_values('Participants', ascending=False)

,Country,Participants,year
196,United States,613,2020
94,Japan,552,2020
10,Australia,477,2020
70,Germany,425,2020
40,China,406,2020
...,...,...,...
192,Tuvalu,2,2020
37,Central African Republic,2,2020
118,Marshall Islands,2,2020
157,Saint Kitts and Nevis,2,2020


In [53]:
# Saving the dataframe to tables in SQL database
import sqlite3

conn = sqlite3.connect('Olympics.db')
c = conn.cursor()

c.execute("""CREATE TABLE 
             OLYMPICS (Olympic Games text,
             Date text, Country text, 
             Athletes int, Teams int, Events int, Year)""")
c.execute("""CREATE TABLE GDP (country text, region text, gdp int)""")
c.execute("""CREATE TABLE INFO (label text, value text, year int)""")
c.execute("""CREATE TABLE 
             WINNERS (rank text, 
             nation text, gold text, 
             silver text, bronze text, 
             total text, year int)""")
c.execute("""CREATE TABLE SPORTS (events text, year int)""")
c.execute("""CREATE TABLE 
             PARTICIPANTS (Country text,
             Participants int, year int)""")

conn.commit()

In [46]:
df_olympics.to_sql('OLYMPICS', conn, if_exists='replace', index = False)
df.to_sql('GDP', conn, if_exists='replace', index = False)
df_info.to_sql('INFO', conn, if_exists='replace', index = False)
df_winners_all.to_sql('WINNERS', conn, if_exists='replace', index = False)
df_sports.to_sql('SPORTS', conn, if_exists='replace', index = False)
df_part_2020.to_sql('PARTICIPANTS', conn, if_exists='replace', index = False)




/opt/conda/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
df_part_2020